### 1. Python: Download a timeseries of daily deaths per country
[Refer to repo](https://github.com/keatmin/covid19-deaths) and the script is in `src/download_csv.py`, if cloned repo make sure 
`download_data.sh` is executable by running `sudo chmod 755 download_data.sh` in terminal. Then `./download_csv.sh` in terminal


```python 

import requests
URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'


def main(): 
    r = requests.get(URL)

    if r.status_code == 200:
        with open('data/covid19_deaths.csv', 'wb') as csv: 
            csv.write(r.content)
        
    else:
        print('Link invalid')
        
        
if __name__=='__main__': 
    main()
```

or run this in terminal `wget https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv`

In [10]:
import pandas as pd 
import psycopg2 as pg
from psycopg2.extras import execute_values 

In [2]:
#Read csv
df = pd.read_csv('../data/covid19_deaths.csv')

Location of the csv is relative to the notebook. 
For a data science project repo, I organize the directories into:
- **data** for data, but also further divided into `00_raw`, `01_inter` and `02_processed`
- **notebooks** for jupyter exploration 
- **src** for script, further divided into `data`, `etl`, `train`, `deploy`

### 2. Python: Convert the table so that each country and each day is a separate row 

In [3]:
df.columns = df.columns.str.lower()
date_cols = list(df.columns[4:])
#drop lat,long for now since we are not using it for this case 
df_long = df.drop(columns=['lat','long'],axis=1)

df_pivoted = pd.melt(df_long.reset_index(),\
                     id_vars=['country/region', 'province/state'],\
                     value_vars=date_cols,var_name='date',\
                     value_name='deaths')

In [4]:
df_pivoted.sample(5)

,country/region,province/state,date,deaths
12277,Luxembourg,NaN,3/9/20,0
16105,France,Guadeloupe,3/24/20,1
2117,China,Gansu,1/30/20,0
5543,Guyana,NaN,2/12/20,0
6186,MS Zaandam,NaN,2/14/20,0


In [11]:
# Calculate change in deaths
df_pivoted['date'] = pd.to_datetime(df_pivoted['date']).dt.date
df_pivoted['deaths_change'] = df_pivoted.groupby('country/region')['deaths'].diff()
#Deal with null values for SQL insert later
processed_df = df_pivoted.where(pd.notna(df_pivoted),None)

In [6]:
processed_df.sample(10)

,country/region,province/state,date,deaths,deaths_change
5752,China,Sichuan,2020-02-13,1,1
1605,China,Hainan,2020-01-28,1,1
5564,Latvia,None,2020-02-12,0,0
16981,Trinidad and Tobago,None,2020-03-27,2,1
12847,Sweden,None,2020-03-11,1,1
17259,Canada,Diamond Princess,2020-03-28,1,1
9546,Afghanistan,None,2020-02-28,0,0
17362,China,Sichuan,2020-03-29,3,3
13724,China,Beijing,2020-03-15,8,2
16646,Iraq,None,2020-03-26,36,7


### 3.  Python: Provide code to upload the table from step 3 into an SQL table named deaths_total

Instruction says **Step 3** but I believe there was a typo and it actually meant **step 2**

Assuming postgres is the database being used

In [ ]:
#example conn
conn = pg.connect(dbname='postgres', port=5432, host='localhost') # add options=f"-c search_path={schema_name}" as kwargs if db is using a non-public schema, which we should
cur = conn.cursor()

In [2]:
drop_deaths_total_table = """DROP TABLE IF EXISTS deaths_total"""  #dropdead

create_deaths_total_table = """CREATE TABLE IF NOT EXISTS deaths_total(
id serial PRIMARY KEY
, country VARCHAR(128)
, province VARCHAR(128)
, date DATE
, deaths INT)"""

In [ ]:
#Drop table
cur.execute(drop_deaths_total)
conn.commit()

In [ ]:
#Create table
cur.execute(create_deaths_total)
conn.commit()

There are multiple ways to do it, the best/fastest way is to use `execute_values` in `psycopg2.extras`

First method using `execute`
```python
for values in processed_df.values: 
    cur.execute("INSERT INTO deaths_total (country,province,date,deaths) VALUES (%s, %s, %s, %s)",(values))
conn.commit()
```
At 1.7s+ 

Second method using `executemany`
```python
cur.executemany("INSERT INTO deaths_total (country,province,date,deaths) VALUES (%s, %s, %s, %s)",(processed_df.values.tolist()))
conn.commit()
```
At 1.7s+ 

Third method using `execute_values`
```python
execute_values(cur,"INSERT INTO deaths_total (country,province,date,deaths) VALUES %s",(processed_df.values.tolist()))
conn.commit()
```
this method is the fastest averaging 400ms 

One other method that is more convenient using pandas `to_sql` but it requires sqlalchemy engine and it's essentially `execute` 
```python
processed_df.to_sql('deaths_total', conn)
```

In [ ]:
# I truncated the table to test with %%timeit
cur.execute("TRUNCATE deaths_total RESTART IDENTITY")
conn.commit()

### 4.  Python: From the data in step 2, calculate the daily change in deaths for each country

- Data cleaning is done at one go above. `processed_df` is the processed data

### 5.  Python: Provide code to upload the table from step 4 into an SQL table named deaths_change_python

In [7]:
death_changes_total = """CREATE TABLE IF NOT EXISTS deaths_change_total(id serial PRIMARY KEY,
country varchar(128),
province varchar(128),
date DATE, 
deaths INT,
change INT)"""

In [8]:
drop_death_change_total = """DROP TABLE IF EXISTS deaths_change_total"""

In [ ]:
cur.execute(drop_death_change_total)
conn.commit()

In [ ]:
cur.execute(death_changes_total)
conn.commit()

Fastest method

```python
execute_values(cur, "INSERT INTO deaths_change_total(country,province,date,deaths,change) VALUES %s", processed_df.values.tolist())
```

### 6. SQL 
- Provide SQL code to calculate the daily change for each country using only the data from `deaths_total` and save it into an SQL table named `deaths_change_sql`

```sql
INSERT INTO deaths_change_sql(country,province,date,deaths,change) 
SELECT country
, province
, date
, deaths
, deaths - LAG(deaths,1) OVER (PARTITION BY country,province ORDER BY date) AS change 
FROM deaths_total
```